# Generate a metadata table with links to inference results

# Set-up

In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np

# GRNs

In [2]:
dataset_names = ["igvf_b01_LeftCortex", "Bridge_Satpathy"]
base_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference"
methods_run = ["aracne", "celloracle", "grnboost2", "scenic", "wgcna"]

In [33]:
run_metadata_paths = {}
metadata_df = pd.DataFrame()
for method in methods_run:
    for dataset_name in dataset_names:
        csv_glob = os.path.join(base_dir, method, "bin", dataset_name, "successful_runs", "**", "*.csv")
        csv_files = glob.glob(csv_glob)
        run_metadata_paths[method] = csv_files
        for csv_file in csv_files:
            df = pd.read_csv(csv_file)
            df["method"] = method
            df["dataset"] = dataset_name
            metadata_df = metadata_df.append(df)
metadata_df = metadata_df.reset_index(drop=True)
metadata_df.head()

/tmp/ipykernel_1256124/4226382858.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1256124/4226382858.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1256124/4226382858.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1256124/4226382858.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1256124/4226382858.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,dataset,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type
0,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# For evert row in the dataframe, I want to check if there exists a file in the "out_dir_path" column called "net.tsv"
def check_net_tsv(row):
    out_dir_path = row["out_dir_path"]
    net_tsv_path = os.path.join(out_dir_path, "net.tsv")
    return os.path.exists(net_tsv_path)
metadata_df["net_tsv_exists"] = metadata_df.apply(check_net_tsv, axis=1)

In [35]:
# How many runs were successful?
metadata_df["method"].value_counts()

aracne        24
celloracle    24
grnboost2     24
scenic        24
wgcna         24
Name: method, dtype: int64

In [36]:
# How many runs have a net.tsv file?
metadata_df[metadata_df["net_tsv_exists"] == True].method.value_counts()

aracne        24
celloracle    24
grnboost2     24
scenic        24
wgcna         24
Name: method, dtype: int64

In [37]:
# How many runs do not have a net.tsv file?
metadata_df[metadata_df["net_tsv_exists"] == False].method.value_counts()

Series([], Name: method, dtype: int64)

# Create a metadata table with all these tsvs

In [39]:
metadata_df[["cells", "genes", "normalization"]] = [i.split("/")[-4:-1] for i in metadata_df["in_file_path"]]
metadata_df["genotype"] = [i.split("/")[-1].split(".")[0].split("-")[-1] for i in metadata_df["in_file_path"]]
metadata_df["grn_name"] = metadata_df["dataset"] + "_" + metadata_df["method"] + "_" + metadata_df["cells"] + "_" + metadata_df["genotype"] + "_" + metadata_df["genes"].astype(str) + "_" + metadata_df["normalization"]
metadata_df

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,dataset,base_grn,...,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists,cells,genes,normalization,genotype,grn_name
0,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,...,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,B6J,igvf_b01_LeftCortex_aracne_balanced_genotype_m...
1,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,...,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,CASTJ,igvf_b01_LeftCortex_aracne_balanced_genotype_m...
2,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,...,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,both,igvf_b01_LeftCortex_aracne_balanced_genotype_m...
3,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,...,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,log1p_cp10k,B6J,igvf_b01_LeftCortex_aracne_balanced_genotype_m...
4,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,igvf_b01_LeftCortex,NaN,...,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,log1p_cp10k,CASTJ,igvf_b01_LeftCortex_aracne_balanced_genotype_m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,836756146_20230830-151919,wgcna,Bridge_Satpathy,NaN,...,NaN,NaN,NaN,signed,True,balanced_genotype_microglia,0.05,log1p_cp10k,CASTJ,Bridge_Satpathy_wgcna_balanced_genotype_microg...
116,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,836756146_20230830-151919,wgcna,Bridge_Satpathy,NaN,...,NaN,NaN,NaN,signed,True,balanced_genotype_microglia,0.05,log1p_cp10k,both,Bridge_Satpathy_wgcna_balanced_genotype_microg...
117,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,836756146_20230830-151919,wgcna,Bridge_Satpathy,NaN,...,NaN,NaN,NaN,signed,True,balanced_genotype_microglia,0.05,pf_log1p_pf,B6J,Bridge_Satpathy_wgcna_balanced_genotype_microg...
118,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,836756146_20230830-151919,wgcna,Bridge_Satpathy,NaN,...,NaN,NaN,NaN,signed,True,balanced_genotype_microglia,0.05,pf_log1p_pf,CASTJ,Bridge_Satpathy_wgcna_balanced_genotype_microg...


In [40]:
metadata_df["output_table_path"] = metadata_df["out_dir_path"].apply(lambda x: os.path.join(x, "net.tsv"))
metadata_df["output_table_exists"] = metadata_df["output_table_path"].apply(lambda x: os.path.exists(x))
metadata_df["output_table_exists"].value_counts()

True    120
Name: output_table_exists, dtype: int64

In [41]:
metadata_out_dir = f"/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/meta_analysis/03Sep23"
os.makedirs(metadata_out_dir, exist_ok=True)

In [42]:
os.chdir(f"/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/meta_analysis/03Sep23")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/meta_analysis/03Sep23'

In [43]:
metadata_df.to_csv("grn_metadata.tsv", index=False, sep="\t")

In [44]:
len(metadata_df)

120

# Post analysis metadata table

# DONE!

---